<a href="https://colab.research.google.com/github/hyunwoo126/activeloop/blob/main/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain==0.0.152 deeplake openai tiktoken pyllamacpp==1.0.7 sentencepiece gpt4all

Download GPT4All (This process might take a while since the file size is 4GB)

In [37]:
import requests
from pathlib import Path
from tqdm import tqdm

local_path = './models/gpt4all-lora-quantized-ggml.bin'
Path(local_path).parent.mkdir(parents=True, exist_ok=True)

url = 'https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized-ggml.bin'

# send a GET request to the URL to download the file.
response = requests.get(url, stream=True)

# open the file in binary mode and write the contents of the response
# to it in chunks.
with open(local_path, 'wb') as f:
    for chunk in tqdm(response.iter_content(chunk_size=8192)):
        if chunk:
            f.write(chunk)

514266it [03:29, 2452.40it/s]


it is time to transform the downloaded file to the latest format. We start by downloading the codes in the LLaMAcpp repository or simply fork it using the following command. (You need to have the git command installed) Pass the downloaded file to the convert.py script and run it with a Python interpreter.

In [ ]:
!rm -rf llama.cpp/
!git clone https://github.com/ggerganov/llama.cpp.git
!python3 llama.cpp/convert.py ./models/gpt4all-lora-quantized-ggml.bin

Import GPT4All

In [39]:
!pip install pyllamacpp==1.0.7

In [1]:
from langchain.llms import GPT4All
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

Setup GPT4All llm instead of OpenAI

In [43]:
!pip install gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.4 MB/s eta 0:00:00


In [3]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# instead of llm = OpenAI(model="text-davinci-003", temperature=0.9)
llm = GPT4All(model="./models/ggml-model-q4_0.bin", callback_manager=callback_manager, verbose=True)
# llm = GPT4All(model="./models/ggml-model-q4_0.bin", verbose=True)


template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What happens when it rains somewhere?"
llm_chain.run(question)

 Question: What happens when it rains somewhere?

Answer: Let's think step by step. When rain occurs, water vapor rises from the Earth’s surface and meets with air in a column above it (weather layer). This process of lifting is called convection. Water droplets or ice crystals form around particles that are found nearer to earth's Surface due to its higher temperature. The smaller, denser water vapor rises upwards and collides with larger raindrops in the atmosphere forming clouds which gives rise to a rainbow (arching rainbows). When these droplets or ice crystals collide they create snowflakes, hailstones etc., If it is too cold for water vapor to condense into liquid form and fall as precipitation, then the cloud would likely dissipate. As a result of this process we get lightning which causes thunder.

" Question: What happens when it rains somewhere?\n\nAnswer: Let's think step by step. When rain occurs, water vapor rises from the Earth’s surface and meets with air in a column above it (weather layer). This process of lifting is called convection. Water droplets or ice crystals form around particles that are found nearer to earth's Surface due to its higher temperature. The smaller, denser water vapor rises upwards and collides with larger raindrops in the atmosphere forming clouds which gives rise to a rainbow (arching rainbows). When these droplets or ice crystals collide they create snowflakes, hailstones etc., If it is too cold for water vapor to condense into liquid form and fall as precipitation, then the cloud would likely dissipate. As a result of this process we get lightning which causes thunder."